In [4]:
import cbpro
import pandas as pd
cb = cbpro.PublicClient()

In [8]:
tickers = ['BTC', 'LTC', 'ZCE']

In [26]:
def RSIcalc(ticker):
    df = pd.DataFrame(cb.get_product_historic_rates(ticker+'-USD', granularity=3600),columns=['time', 'low', 'high', 'open', 'close', 'volume'])
    df['MA200'] = df['close'].rolling(window=200).mean()
    df['price change'] = df['close'].pct_change()
    df['Upmove'] = df['price change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['price change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['avg Up'] = df['Upmove'].ewm(span=19).mean()
    df['avg Down'] = df['Downmove'].ewm(span=19).mean()
    df = df.dropna()
    df['RS'] = df['avg Up']/df['avg Down']
    df['RSI'] = df['RSI'].apply(lambda x: 100-(100/(x+1)))
    df.loc[(df['close'] > df['MA200']) & (df['RSI'] < 30), 'Buy'] = 'Yes'
    df.loc[(df['close'] < df['MA200']) | (df['RSI'] > 30), 'Buy'] = 'No'
    return df

In [27]:
RSIcalc(tickers[0])

KeyError: 'RSI'

In [1]:
from slack import send_message
from coinbase import get_holdings
send_message(get_holdings())


'Current holdings in coinbase pro \n\t==> USD $8.0\n\t==> USDC 0.00007897663225\n\t==> ZEC 0.01363376\n'